In [14]:
import re
import string
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\moham\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\moham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
def preprocessing(article):
    lines = article.split(".")   # splits the whole article into lines
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    article_preprocessed = []    # list that contains the main sentences after being preprocessed 
    for line in lines:
        line_preprocessed = []
        words_in_line = line.split()
        for word in words_in_line:
            if (word not in stopwords_english and word not in string.punctuation):   # make sure word is not a stop word
                                                                                     # and not a punctuation 
                word_stemmed = stemmer.stem(word)  
                line_preprocessed.append(word_stemmed)
        article_preprocessed.append(line_preprocessed)
    return article_preprocessed

In [16]:
with open("article_1.txt", "r", encoding="utf8") as file:
    contents = file.read()
    article_preprocessed = preprocessing(contents)

In [17]:
print(article_preprocessed)

[['the', '“a”', 'uaw', 'long', 'carri', 'elabor', 'set', 'meanings,', 'econom', 'social'], ['root', 'vehicl', 'factori', 'upper', 'midwest,', 'unit', 'automobil', 'worker', 'organ', 'hundr', 'thousand', 'blue-collar', 'worker', 'america’', 'import', 'industri', 'onset', 'second', 'world', 'war'], ['that', 'war', 'gener', 'aircraft', 'industri', 'jobs,', 'mani', 'held', 'femal', 'rosi', 'rivet', 'airplan', 'frame'], ['by', 'end', 'war,', 'mani', 'worker', 'join', 'uaw', 'alongsid', 'worker', 'farm', 'equip', 'industry,', 'give', 'union', 'new', 'full', 'name:', 'unit', 'automobile,', 'aircraft,', 'agricultur', 'implement', 'worker'], ['(aircraft', 'would', 'later', 'chang', 'aerospac', 'cold', 'war', 'missil', 'race'], ['now,', 'unofficially,', '“a”', 'also', 'stand', 'academ', 'workers,', '48,000', 'strike', 'univers', 'california'], ['though', 'formal', 'chang', 'uaw', 'name', 'sinc', '1962,', 'doubt', 'academi', 'becom', 'dynam', 'success', 'venu', 'union', 'recent', 'year'], ['for',